In [48]:
import pandas as pd
import nltk
from nltk.corpus import stopwords 
from sklearn.preprocessing import scale
import numpy as np
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from matplotlib import pyplot as plt
import pickle
import time
start = time.time()
import re
# Useful libs
from sklearn.datasets import fetch_20newsgroups
import pickle
import pandas as pd
import numpy as np
# STTM lib from Github
from gsdmm import MovieGroupProcess
#from topic_allocation import top_words, topic_attribution
import spacy
nlp = spacy.load('en_core_web_sm')

In [49]:
#Import and expand on stopwords
import spacy
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use', "amp"])

In [50]:
#define functions to remove stopwords, make bigrams, make trigrams, and lemmatize words
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
def remove_stopwords_tokenized(doc):
    return [word for word in simple_preprocess(str(doc)) if word not in stop_words]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [51]:
#Define function to put sentences to words
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

In [54]:
#Define stemming function
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer(language='english')
def stem(text_list):
    return [stemmer.stem(item) for item in text_list]

In [55]:
#Define function to remove "amp" which stands for ampersands in the text data
amp = ["amp"]
def remove_amp_tokenized(doc):
    return [word for word in simple_preprocess(str(doc)) if word not in amp]

In [56]:
#Load data
with open('KP_Tweets_For_SSTM_Topic_Modeling.pickle', "rb") as f:
    tweets = pickle.load(f)

In [58]:
#Tokenize tweets
data = tweets.clean_translation_tokenized.values.tolist()
data_bis = [item for sublist in data for item in sublist]
#len(data)

200

In [59]:
#Remove unique tokens
from collections import Counter
duplicates = {k for k, v in Counter(data_bis).items() if v > 1}
for i in range(len(data)):
    data[i] = [el for el in data[i] if el in duplicates]
#len(result)

In [60]:
#Create input to model
docs = data
vocab = set(x for doc in docs for x in doc)
n_terms = len(vocab)

print("Vocab size:", n_terms)
print("Number of documents:", len(docs))


Vocab size: 288
Number of documents: 200


In [61]:
# Train STTM model
# Init of the Gibbs Sampling Dirichlet Mixture Model algorithm
# K = number of potential topic (which we don't know a priori)
# alpha = 
# beta = 
# n_iters = number of iterations to 
mgp = MovieGroupProcess(K=25, alpha=0.1, beta=0.1, n_iters=10)
tffuytfy = mgp.fit(docs, n_terms)


In stage 0: transferred 134 clusters with 5 clusters populated
In stage 1: transferred 76 clusters with 5 clusters populated
In stage 2: transferred 61 clusters with 5 clusters populated
In stage 3: transferred 61 clusters with 5 clusters populated
In stage 4: transferred 61 clusters with 5 clusters populated
In stage 5: transferred 62 clusters with 5 clusters populated
In stage 6: transferred 63 clusters with 5 clusters populated
In stage 7: transferred 48 clusters with 5 clusters populated
In stage 8: transferred 61 clusters with 5 clusters populated
In stage 9: transferred 62 clusters with 5 clusters populated


In [62]:
#Show clusters and top words per cluster
doc_count = np.array(mgp.cluster_doc_count)
print('Number of documents per topic :', doc_count)
print('*'*20)# Topics sorted by the number of document they are allocated to
top_index = doc_count.argsort()[-10:][::-1]
print('Most important clusters (by number of docs inside):', top_index)
print('*'*20)# Show the top 8 words in term frequency for each cluster 

Number of documents per topic : [46 49 25 36 44]
********************
Most important clusters (by number of docs inside): [1 0 4 3 2]
********************


In [80]:
#Store word clusters in list
result = []

for i in range(len(mgp.cluster_word_distribution)):
    cluster_dict = mgp.cluster_word_distribution[i]
    result.append(sorted(cluster_dict, key=cluster_dict.get, reverse=True)[:8])

In [82]:
#Convert word lists to dataframe
result = pd.DataFrame(result)

In [83]:
#Convert number of documents per cluster to dataframe
doc_count = pd.DataFrame(doc_count)

In [84]:
#Merge and save the dataframe of results
result = doc_count.merge(result, left_index=True, right_index=True)
result.columns = ["tweets", "word1", "word2", "word3", "word4", "word5", "word6", "word7", "word8"]
with open("KP_TOPICS_FINAL.pickle", "wb") as f:
    pickle.dump(result, f)

In [88]:
#Define a function to assign a topic to each tweet
def topic(x):
    (y,z) = mgp.choose_best_label(x)
    return y

In [89]:
#Define a function to assign percent fit for each tweet per topic
def percent(x):
    (y,z) = mgp.choose_best_label(x)
    return z

In [90]:
#Assign topics to tweets
tweets["topic"] = tweets["clean_translation_tokenized"].apply(lambda x: topic(x))

In [91]:
#Assign percents to tweets
tweets["percent"] = tweets["clean_translation_tokenized"].apply(lambda x: percent(x))

In [ ]:
#Save tweets with topics assigned
with open("kp_tweets_topics_assigned.pickle", "wb") as f:
    pickle.dump(tweets, f)